In [1]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
env = GridworldEnv()

$ \\
v_{k+1}(s)=\sum_{a{\in}A}{\pi}(a|s)(R_{s}^{a}+{\gamma}\sum_{s{\in}S}P_{ss^{\prime}}^{a}v_{k}^{s^{\prime}} \\
\boldsymbol{v}^{k+1}=R^{\pi}+{\gamma}P^{\pi}\boldsymbol{v}^{k} \\
R_{s}^{\pi}=\sum_{a{\in}A}{\pi}(a|s)R_{s}^{a} \\
P_{s}^{\pi}=\sum_{a{\in}A}{\pi}(a|s)P_{ss^{\prime}}^{a} \\
$


In [3]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    V = V.reshape((-1,1))
    
    R_pi = np.array([np.sum([policy[s][a] * env.P[s][a][0][2]
                             for a in range(env.nA)])
                     for s in range(env.nS)])
    R_pi = R_pi.reshape((-1,1))

    P_pi = np.array([
                        [
                             np.sum([policy[s_from][a]
                             for a in range(env.nA)
                             if env.P[s_from][a][0][1]==s_to])
                             
                         for s_to in range(env.nS)]
                     for s_from in range(env.nS)])

    while True:
        # TODO: Implement!
        V_new = R_pi + discount_factor * np.matmul(P_pi, V)
        if np.max(np.abs(V_new - V)) < theta:
            V = V_new
            break
        V = V_new
    return V.flatten().tolist()

In [4]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

In [5]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

In [6]:
import timeit
n = 100
t = timeit.timeit("policy_eval(random_policy, env)", globals=globals(), number = n) / n
print("avg time: {}".format(t))

avg time: 0.0038263267799629828
